In [159]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
pallet = pallet.drop(['lot_code', 
                      'tran_type', 
                      'final_pallet_code', 
                      'warehouse_facility_id',
                      'source_system_id'], axis=1)
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

## Plan de campagne:

- Process the weather data into workable data for the algo
    - DONE
- Calculate on a minute-to-minute basis how many products are 'new' in the warehouse.
    - Sonja
- Try to estimate on a minute-to-minute basis how many doors are open at any point in time.
    - Suzanne
- Optimaliseren van de Random Forest Parameters.
    - Rob
- NaN values automatiseren.
    - Rob

In [160]:
# for data visualization
import math, datetime, time, random
import matplotlib.pyplot as plt
import missingno

In [163]:
def weekend(x):
    if x['weekday'] > 4:
        return True
    return False
    

def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['weekend'] = df.apply(weekend, axis=1)
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df


#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()


#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame
base_df = pd.concat([base_df, base_weather], axis=1)

# Drop all NaN values
#base_df.dropna(subset=['demand_kW', 'Temperature'])

In [207]:
dummy_df = base_df.copy()

dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_normalized_df = dummy_df.copy()
dummy_normalized_df = dummy_normalized_df.dropna(subset=['demand_kW'], axis=0)
#dummy_df.demand_kW = dummy_df.demand_kW.interpolate()
dummy_normalized_df

,demand_kW,year,month,weekday,weekend,day,minute,Relative Humidity,Temperature
42,2064.101392,2018.0,12.0,0.0,False,31.0,15.0,61.27,46.4
45,1874.002081,2018.0,12.0,0.0,False,31.0,30.0,61.27,46.4
48,1988.168511,2018.0,12.0,0.0,False,31.0,45.0,65.60,44.6
52,2022.795943,2018.0,12.0,0.0,False,31.0,0.0,65.60,44.6
55,1986.981872,2018.0,12.0,0.0,False,31.0,15.0,65.60,44.6
...,...,...,...,...,...,...,...,...,...
459926,2217.847000,2021.0,10.0,0.0,False,11.0,3.0,NaN,NaN
459927,2184.012000,2021.0,10.0,0.0,False,11.0,4.0,NaN,NaN
459928,2159.482000,2021.0,10.0,0.0,False,11.0,5.0,87.75,57.2
459929,2145.155000,2021.0,10.0,0.0,False,11.0,6.0,NaN,NaN


In [208]:
def normalize_column(df, columnname):
    """Function which returns a Dataframe where the given column is normalized through min-max normalization."""
    df[f'{columnname}_normalized'] = (df[columnname] - df[columnname].min()) / (df[columnname].max() - df[columnname].min())
    return df.drop([columnname], axis=1)

def add_one_hot_encoder(df, colname):
    """
    Function which returns a DataFrame where the given column has been removed and replaced by
    one-hot-encoding columns for each value in the original column.
    """
    onehot = pd.get_dummies(df[colname], prefix=colname)
    return df.drop(colname, axis=1).join(onehot)

def interpolate_column(df, colname):
    df[f'{colname}_interpolated'] = df[colname].interpolate(method='linear')
    return df.drop([colname], axis=1)

interpolate = ['Temperature']
to_normalize = ['Relative Humidity', 'Temperature_interpolated']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    dummy_normalized_df = interpolate_column(dummy_normalized_df, x)
for x in to_normalize:
    dummy_normalized_df = normalize_column(dummy_normalized_df, x)
for x in add_one_hot_encoding:
    dummy_normalized_df = add_one_hot_encoder(dummy_normalized_df, x)


dummy_normalized_df.columns


Index(['demand_kW', 'weekend', 'day', 'minute', 'Relative Humidity_normalized',
       'Temperature_interpolated_normalized', 'weekday_0.0', 'weekday_1.0',
       'weekday_2.0', 'weekday_3.0', 'weekday_4.0', 'weekday_5.0',
       'weekday_6.0', 'year_2018.0', 'year_2019.0', 'year_2020.0',
       'year_2021.0', 'month_1.0', 'month_2.0', 'month_3.0', 'month_4.0',
       'month_5.0', 'month_6.0', 'month_7.0', 'month_8.0', 'month_9.0',
       'month_10.0', 'month_11.0', 'month_12.0'],
      dtype='object')

# Model Building

First we build the skeleton:
- Divide into train/test
- Set target column
- Get Accuracy

In [209]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor



#Define train, test sets\n",
train, test = train_test_split(dummy_normalized_df, shuffle=True)
X_train = train.copy().drop(['demand_kW'], axis=1)
Y_train = train['demand_kW']
X_test = test.copy().drop(['demand_kW'], axis=1)
Y_test = test['demand_kW']

In [113]:
""" Support Vector Regression 
svm = SVR()
svm.fit(X_train, Y_train)
acc = svm.predict(X_test)
result = r2_score(list(Y_test), acc)
result = 0.01039275644409332 """

' Support Vector Regression \nsvm = SVR()\nsvm.fit(X_train, Y_train)\nacc = svm.predict(X_test)\nresult = r2_score(list(Y_test), acc)\nresult = 0.01039275644409332 '

In [210]:
# Random Forest Regression
rfr = RandomForestRegressor(
    verbose=1,
    n_estimators=100
    )
rfr.fit(X_train, Y_train)
rfr_acc = rfr.predict(X_test)
print(r2_score(list(Y_test), rfr_acc))
print(mean_absolute_error(list(Y_test), rfr_acc))

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [139]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train, Y_train)
knn_acc = knn.predict(X_test)
print(r2_score(list(Y_test), knn_acc))
print(mean_absolute_error(list(Y_test), knn_acc))

0.7798950705342219
147.53241085002927


In [40]:
print(mean_absolute_error(list(Y_test), rfr_acc))

116.73017084907966


In [138]:
featureImportance = rfr.feature_importances_
for x in range(len(featureImportance)):
    print(f'{dummy_normalized_df.columns[1:][x]} has a feature importance of: {featureImportance[x]}.')

weekend has a feature importance of: 0.014407613347680535.
day has a feature importance of: 0.06943921054621234.
minute has a feature importance of: 0.03251696935106511.
Relative Humidity_normalized has a feature importance of: 0.07726219700494129.
Temperature_normalized has a feature importance of: 0.5780663251638286.
weekday_0.0 has a feature importance of: 0.005648062584076138.
weekday_1.0 has a feature importance of: 0.0047622231500451.
weekday_2.0 has a feature importance of: 0.007272731962286338.
weekday_3.0 has a feature importance of: 0.003857524750635072.
weekday_4.0 has a feature importance of: 0.004603091811849178.
weekday_5.0 has a feature importance of: 0.0020071204895514517.
weekday_6.0 has a feature importance of: 0.003762709476702905.
year_2018.0 has a feature importance of: 1.8351679462513476e-05.
year_2019.0 has a feature importance of: 0.02009643291555088.
year_2020.0 has a feature importance of: 0.05658644464397794.
year_2021.0 has a feature importance of: 0.0230881

In [14]:
print(rfr_parae)

demand_kW


# From here, model testing. Delete at your own discretion.

In [117]:
from sklearn.preprocessing import OneHotEncoder

In [127]:
X = OneHotEncoder()
dummy_df['weekday']
temp = pd.get_dummies(dummy_df['weekday'], prefix='weekday')


,weekday_0.0,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,weekday_6.0
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
116228,1,0,0,0,0,0,0
116229,1,0,0,0,0,0,0
116230,1,0,0,0,0,0,0
116231,1,0,0,0,0,0,0


In [62]:
from sklearn import preprocessingx

### Failed attempt to automate parameter search+

In [157]:
import time
n_estimators = [10, 20, 50, 100, 200, 500]
best = [0, 10000000000]
for x in n_estimators:
    start = time.time()
    print(f"Starting with {x} estimators.")
    random_forest = RandomForestRegressor(n_estimators=x)
    print('Fitting model...')
    random_forest.fit(X_train, Y_train)
    random_forest_acc = random_forest.predict(X_test)
    stop = time.time()
    mae = mean_absolute_error(list(Y_test), random_forest_acc)
    print(r2_score(list(Y_test), random_forest_acc))
    mean_absolute_error(list(Y_test), random_forest_acc)
    print(f'Time taken: {stop - start} seconds. \n')
    if mae < best[1]:
        best = [x, mae]
print(f'{best[0]} is the best n_estimator, with a mean absolute error of {best[1]}.' )

Starting with 10 estimators.
Fitting model...
0.8561311838951838
Time taken: 5.177995681762695 seconds. 



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [140]:
dat1 = [X_train, Y_train, X_test, Y_test]
def rf_optimizer(data, vals):
    base = {
        'n_estimators': [10, 50, 100],
        'max_depth': [2, 4, 6],
        'min_samples_split': [2, 4, 8]
        }

# loop through the hyperparameters and fit the model
for params in param_grid:
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

In [150]:
test = 'n_estimators'
rfr = RandomForestRegressor()

In [153]:
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [2, 4, 6],
    'min_samples_split': [2, 4, 8]
}

# loop through the hyperparameters and fit the model
for params in param_grid:
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    print(**params)

TypeError: sklearn.ensemble._forest.RandomForestRegressor() argument after ** must be a mapping, not str